# Bagging vs boosting

Le but des méthodes d'ensemble est de combiner les prédictions de plusieurs algorithmes. 
plusieur modele de machine learning pour en creer un plus puissant (plus précis, moins de variance...)

Les méthodes d'ensembles consiste en une collection de prédiction que l'on rassemble (par exemple moyenne) pour avoir une prédiction finale. <br>
Nous verrons ici 2 types de méthode d'ensemble:
* les méthodes de bagging (avec la forest aléatoire)
* les métodes de boosting (gradient boosting tree)

Les méthodes de bagging : on construit un ensemble d'estimateurs, de predictions, de learners indépendants que l'on combinent (moyenne, mode, medianne...)

Le méthodes de boosting :  on construit des estimateurs, des prédicteurs, des learners de manière sequentielle. L'estimateurs, le prédicteur, le learner subséquent apprend des erreurs des estimateurs,      prédicteurs,learners précédants.

# Arbre de décision

Un arbre de décision est une classification par série de test successif. Les arbres sont le résultat d'algorithme CART (classification and regression trees)

In [2]:
import mglearn
mglearn.plots.plot_animal_tree()
#display(tree)
from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris()
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(iris.data, iris.target)
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
#graph 

In [6]:
from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris()
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(iris.data, iris.target)
import graphviz 
rep = open("/home/fabien/Bureau/COURSPYTHON/ensemble/tree.png",'w')
dot_data = tree.export_graphviz(clf, out_file=rep, 
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True) 
rep.close()
#graph = graphviz.Source(dot_data)  

In [13]:
from IPython.display import SVG
from graphviz import Source
#graph.format = 'png'
graph = Source(tree.export_graphviz(clf, out_file=None,feature_names=iris.feature_names,class_names=iris.target_names))
#SVG(graph.pipe(format='svg'))
graph.format = 'png'
graph.render('dtree',view=True)

'dtree.png'

Ci-dessus, un arbre de décision de profondeur 3 permettant de prédire l'espece des iris.

# Random forest (forest aléatoire)

On se donne un ensemble d'apprentissage $\{(x^{1},y^{1}),\dots,(x^{n},y^{n})\}$. $x^{i}\in \mathbb{R}^{m}$.
* Dans les problème de classification :$y^{i} \in \{-1,1\}$. 
* Dans les problème de régréssion : $y^{i}\in \mathbb{R}$

L'espace des features est de dimention m, on a m features.<br>
Avec la stratégie one versul all ou bien la strategie one vs one les forest aléatoire traite de problème multiclasse.

Une forest aléatoire est un ensemble d'arbre de décision indépendants. Pour construire la foret aléatoire, 
* on doit choisir le nombre d'arbre (n_estimator dans les algorithmes RandomForestClassifier et RandomForestRegressor) 
* le nombre de features maximum que l'on utilise lors de la division d'un neux (max_feature=p,p<=m). Lors du scindage d'un neux, la division qui est choisit est la meilleur parmis le sous ensemble alatoire de p features.
* la fonction qui mesure la qualité des divisions des neux de chaque arbre 
     * criterion ='gini', 'entropy' pour la classification
     *           ='mse', 'mae' pour la regression
* La profondeur maximal de chaque arbre (max_depth)
* Les autres citéres des arbres de décision

Chaque arbre est construit avec l'algorithme CART

Pour faire une prédiction avec un algorithme de la foret aléatoire
* On fait une prediction pour chaque arbre de la foret
* Ensuite :
  * Pour la régréssion, la moyenne des résultats précédants produit la prédiction finale
  * Dans le cas d'une classification, la prédiction finale est le mode des résultats précédant

## Algorithme de la forest aléatoire
Entrées:
* $x$, l'observation à prévoir
* $\{(x^{1},y^{1}),\dots,(x^{n},y^{n})\}$ l'échantillon d'apprentissage
* B le nombre d'arbre (n_estimator en python)
* $p \in \mathbb{N}^{*}$ le nombre le nombre de features candidats pour découper un noeud (max_feature en python)

Pour m=1 à B:
* tirer un échantillon bootstrap de $\{(x^{1},y^{1}),\dots,(x^{n},y^{n})\}$
* Construire un arbre CART sur cet échantillon bootstrap, chaque coupure est sélectionnée en minimisant la fonction de coût de CART sur un ensemble de p features choisis au hasard parmi les m. On note $h_{m}$ l'arbre construit.

Sortie
* l'estimateur $F(x)=\frac{1}{B}\sum_{m=1}^{B}h_{m}(x)$ pour une régression
* l'estimateur $F(X)=mode(h_{1}(X),\dots,h_{m}(X)$ pour une classification

# Gradient Boosting Tree

## Fonctionnement du Gradient Boosting

Gradient boosting implique 3 éléments:
* une fonction de perte à optimiser.
* un "weak learner" pour faire des prédiction.
* un modéle additife additionnant les weak learners dans le but de minimiser la fonction de perte

L'algorithme du gradient boosting tree permet de traiter de problème de classification binaire, classification multilcasse et de régression

## Modele additife

L'algorithme du gradient boosting tree produit une fonction de la forme:
$$F(x)=\sum_{m=1}^{M}\gamma_{m}h_{m}(x)$$
ou les $h_{m}$ sont des arbre de décision de taille fixe. Ces arbre sont qualifé de "weak learners" tandis que $F(x)$ est qualifié de strong learner. On parle de modele aditif car $F(x)$ est une somme.

## Fonction de perte 
La fonction de perte utilisée depends du type de problème (classification  ou régression) que l'on doit résoudre. Les fonctions de perte doivent être différentiable. 
En sklearn, on utilisera les fonctions de pertes suivantes:
* logarithmic loss:  classifcation à 2 classes $L(Y,f(x))=\ln\left(1+e^{-2Yf(x)}\right)$
* Exponential loss (classification à 2 classes $L(Y,f(x))=e^{-yf(x)}$
* squared error (regression) $L(Y,f(x))=(Y-f(x))^{2}$



## L'algorithme du gradient 
On se donne un échantillon $\{(x^{1},y^{1}),\dots,(x^{n},y^{n})\}$ $x^{i}\in \mathbb{R}^{m}$.
* Dans les problème de classification :$y^{i} \in \{-1,1\}$. 
* Dans les problème de régréssion : $y^{i}\in \mathbb{R}$

L'algorithme produit une fonction:
$$F(x)=\sum_{i=1}^{M}\gamma_{m}h_{m}(x)$$
ou les $h_{m}$ sont des arbres (décision et regression) de taille fixe. F est construit à l'aide de M itérations (M arbres):
$$F_{m}(x)=F_{m-1}(x)+v\gamma_{m}h_{m}(x) $$
ou le nouvel arbre $h_{m}$ est solution du problème:
$$h_{m}=\underset{h}{argmin}\sum_{i=1}^{n}L(y^{i},F_{m-1}(x^{i})+h(x^{i})) $$
Le modèle initial $F_{0}$ est specifique au problème, dans le cas d'un problème de régression on utilise la moyenne des valeurs cibles. L désigne une fonction de perte. $v$ est le paramètre du taux d'apprentissage (learning rate).
On a:

On a:
$$F_{m}(x)=F_{m-1}(x)-\gamma_{m}\sum_{i=1}^{n}\nabla_{F}(y^{i},F_{m-1}(x^{i})) $$
ou
$$\gamma_{m}=\underset{\gamma}{argmin}\sum_{i=1}^{n}L(y^{i},F_{m-1}(x^{i})-\gamma \frac{\partial L(y^{i},F_{m-1}(x^{i}))}{\partial F_{m-1}(x^{i})})$$
Les algorithmes pour la  regression et la classification differ dans la fonction de perte L que l'on utilise.

## Les paramètres du gradient boosting tree
* Le nombre d'arbre impliqué se controle par le paramêtre n_estimators
* La profondeur de chaque est controlé par le paramêtre max_depth
* Le taux d'apprentissage corespond au paramêtre learning_rate
* max_features : le nombre de features que l'on utilise pour le meilleur split 

# Exemples classification
Nous allons utiliser les donnée digit. la variable à predire à 9 classes.

Importation des données

In [4]:
from sklearn.datasets import load_digits
digit = load_digits()
X = digit['data']
Y = digit['target']

Division en échantillon d'apprentissage et de test:

In [5]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25\
                                                 ,random_state=1998)

## Classification de donnée digit avec une forest aléatoire

In [6]:
from sklearn.ensemble import RandomForestClassifier

clf1 = RandomForestClassifier(n_estimators=60,max_depth=6,\
                             max_features=0.6,random_state=1998)

On entraine le forest aléatoire clf1

In [7]:
clf1.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=0.6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
            oob_score=False, random_state=1998, verbose=0,
            warm_start=False)

On fait la prédiction sur X_test

In [10]:
Y_pred = clf1.predict(X_test)

On calcul la precision du classiffieur clf1

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.94

## Classification de donnée digit avec gradient boosting tree

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
clf2 = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,\
                                  n_estimators=100,\
                                  max_depth=4,random_state=1998)

On entraine le gradient boosting:

In [13]:
clf2.fit(X_train,Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=1998, subsample=1.0, verbose=0,
              warm_start=False)

On fait une prédiction sur X_test

In [14]:
Y_pred = clf2.predict(X_test)

On calcule la précision du gradient boosting

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.9577777777777777

# Exemple régression
Nous allons prédire la température critique des donnée superconductivity. Les données proviennent de https://archive.ics.uci.edu/ml/datasets/Superconductivty+Data

Importation des données, dummiefication des variables

In [16]:
import pandas as pd
rep = "C:/Users/IFDU1270/Desktop/DATABASE/superconduct/train.csv"
SUPERCONDUCT = pd.read_csv(rep)

On creer Y correspondant à la valeur cible et X correspondant au features

In [35]:
import pandas as pd
Y = SUPERCONDUCT['critical_temp']
X = SUPERCONDUCT.drop(columns=['critical_temp'],axis=1)

X_ADUM = pd.get_dummies(X['number_of_elements'].astype(str),prefix='elements')
X = X.drop(columns=['number_of_elements'],axis=1)
X = pd.concat([X_ADUM,X],axis=1)

On divise en échantillon d'apprentissage et de test:

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,\
                                                random_state=1998)

## RandomForestRegressor

On créer un modele de RandomForestRegressor:

In [55]:
from sklearn.ensemble import RandomForestRegressor
reg1 = RandomForestRegressor(n_estimators=110,max_depth=25,\
                             random_state=1998,max_features='auto')

On lance la phase d'apprentissage:

In [56]:
reg1.fit(X_train,Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=110, n_jobs=1,
           oob_score=False, random_state=1998, verbose=0, warm_start=False)

On fait une prédiction sur X_test

In [57]:
Y_pred = reg1.predict(X_test)

On calcule le score R2 de la prédiction:

In [58]:
from sklearn.metrics import r2_score
r2_score(Y_test,Y_pred)

0.9218914641594653

## GradientBoostingRegressor

In [65]:
from sklearn.ensemble import GradientBoostingRegressor
reg2 = GradientBoostingRegressor(learning_rate=0.05, n_estimators=200,\
                                max_depth=5)

On lance l'étape d'aprentissage

In [66]:
reg2.fit(X_train,Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

On predit sur les données X_test

In [62]:
Y_pred = reg2.predict(X_test)

On calcule le score r2:

In [64]:
from sklearn.metrics import r2_score
r2_score(Y_test,Y_pred)

0.9105603420543338

## Référence 
Voici quelques référence:
https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/